In [1]:
from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import subprocess
import soundfile as sf

In [2]:
files = glob('parlimen-24k-chunk_processed/**/*/*.json', recursive = True)

len(files)

3399

In [3]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [4]:
threshold = -18

def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def new_path_lang(f):
    f = f.replace('.mp3', '.language')
    f = f.replace('_processed/', '_processed_language/')
    return f

def chunk(alignment, reject = threshold, minimum_length = 2.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls
        
def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

def detect_extra(word):
    word = clean(word)
    return word in [
        'eh', 'ehh', 'oh', 'ohh', 'uhm', 'uhmm',
        'um', 'em', 'emm', 'ah', 'ha', 'ok', 'okay',
        'so', 'yes', 'no', 'ah', 'aa', 'so', 'uh', 'um', 'eh', 'ha', 'oh', 'ye', 'haa',
        'oi', 'ya', 'leh', 'lah', 'haiya', 'hoi', 'haha', 'hahaha',
        'then', 'it s'
    ]

from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [5]:
!rm -rf parliament-chunk
!mkdir parliament-chunk

In [6]:
def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
                
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            for a in align:
                if detect_extra(a['text']):
                    a['score'] = 0.0
                
            scores = [a for a in align if a['score'] <= threshold]
            if not len(scores):
                continue
            
            chunks = chunk(align)
            if len(chunks):
                y, sr = sf.read(audio_path)
                for no, c in enumerate(chunks):
                    if len(c) == len(align):
                        continue
                    try:
                        
                        words = [c_['text'] for c_ in c if len(c_['text']) <= 1]
                        if (len(words) / len(c)) > 0.5:
                            print(c)
                            continue
                        
                        skip = False
                        
                        for c_ in c:
                            if (c_['end'] - c_['start']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                        
                        for i in range(1, len(c), 1):
                            if (c[i]['start'] - c[i - 1]['end']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                            
                        t = ' '.join([c_['text'] for c_ in c])
                        start = c[0]['start']
                        end = c[-1]['end']
                        
                        a = audio_path.replace('/', '_').replace('.mp3', '') 
                        a = os.path.join('parliament-chunk', f'{a}_{no}.mp3')
                        if not os.path.exists(a):
                            sf.write(a, y[int(sr * start): int(sr * end)], sr)

                        data.append({
                            'audio': a,
                            'transcription': t,
                        })
                    except:
                        pass
    
    return data

In [7]:
d = loop((files[-1:], 0))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.99s/it]


In [8]:
len(d)

190

In [9]:
d[-20]

{'audio': 'parliament-chunk/parlimen-24k-chunk_processed_parlimen-24k-LANGSUNG ： Persidangan Dewan Rakyat 3 Ogos 2022 ｜ Sesi Petang [9hnsHzRGtlY]_001_parlimen-24k-LANGSUNG ： Persidangan Dewan Rakyat 3 Ogos 2022 ｜ Sesi Petang [9hnsHzRGtlY]_001_252_0.mp3',
 'transcription': 'Kederaan yang membawa barang-barang pergi keluar'}

In [10]:
import IPython.display as ipd
ipd.Audio(d[-20]['audio'])

In [11]:
data = mp.multiprocessing(files, loop, cores = 10)

 12%|██████████                                                                              | 39/339 [05:35<43:20,  8.67s/it]

[{'start': 2.92, 'end': 3.94, 'text': '51', 'score': 0.0}, {'start': 3.94, 'end': 4.0, 'text': '5', 'score': -7.700920104980469e-05}, {'start': 4.6, 'end': 4.84, 'text': 'kurungan', 'score': -4.288383483886719}, {'start': 4.86, 'end': 4.88, 'text': '1', 'score': 0.0}, {'start': 4.94, 'end': 6.02, 'text': 'A', 'score': 0.0}]


 15%|█████████████▏                                                                          | 51/339 [06:17<39:09,  8.16s/it]

[{'start': 1.66, 'end': 1.74, 'text': 'dari', 'score': -7.1641693115234375}, {'start': 1.96, 'end': 2.1, 'text': 'Great', 'score': -8.032257080078125}, {'start': 2.14, 'end': 2.46, 'text': 'Britain,', 'score': -12.228103637695312}, {'start': 3.02, 'end': 3.02, 'text': 'C', 'score': 0.0}, {'start': 3.34, 'end': 3.34, 'text': 'O', 'score': 0.0}, {'start': 3.96, 'end': 3.96, 'text': 'L', 'score': 0.0}, {'start': 4.08, 'end': 4.08, 'text': 'O', 'score': 0.0}, {'start': 4.32, 'end': 4.32, 'text': 'U', 'score': 0.0}, {'start': 4.58, 'end': 4.72, 'text': 'R.', 'score': -0.23968124389648438}]


 61%|█████████████████████████████████████████████████████▍                                 | 208/339 [27:50<15:32,  7.12s/it]

[{'start': 16.58, 'end': 16.98, 'text': '2', 'score': 0.0}, {'start': 18.0, 'end': 18.34, 'text': 'kurungan', 'score': -1.6227827072143555}, {'start': 18.36, 'end': 18.88, 'text': '9', 'score': 0.0}]


100%|███████████████████████████████████████████████████████████████████████████████████████| 339/339 [39:14<00:00,  6.95s/it]


In [12]:
len(data)

327013

In [13]:
from collections import defaultdict

uniques = defaultdict(int)
for d in tqdm(data):
    uniques[d['audio'].split('_processed')[0]] += 1
    
uniques

100%|████████████████████████████████████████████████████████████████████████████| 327013/327013 [00:00<00:00, 2936204.05it/s]


defaultdict(int, {'parliament-chunk/parlimen-24k-chunk': 327013})

In [14]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,audio,transcription
0,parliament-chunk/parlimen-24k-chunk_processed_...,Segala puji bagi Allah dibuka majlis dewan rak...
1,parliament-chunk/parlimen-24k-chunk_processed_...,"yang bersifat maha hikmat, limpah keadilan, ya..."
2,parliament-chunk/parlimen-24k-chunk_processed_...,"Mujuk kepada perkara teratai, saya ingin menar..."
3,parliament-chunk/parlimen-24k-chunk_processed_...,Pertanian dan Industri Makan berhubung
4,parliament-chunk/parlimen-24k-chunk_processed_...,"Yang Berhormat Rumpin. Sekarang, saya jemput Y..."


In [15]:
df.to_parquet('verify-text-chunk-parliament.parquet')

In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="verify-text-chunk-parliament.parquet",
    path_in_repo="data/parliament_chunk-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/c5a7937e39dff59f1233e47ece68de758add7269', commit_message='Upload data/parliament_chunk-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='c5a7937e39dff59f1233e47ece68de758add7269', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)